In [3]:
import pandas as pd
import re
import numpy as np

# Please run this code with the .csv files in 'cleaned' folder and this .ipynb would return a list of 'xxx_538.csv' files (giving the grade to the poll data according to the 538 webiste'

In [5]:
pollster = pd.read_csv('pollster.csv')
pollster.head()
for i in range(0,len(pollster.iloc[:,0])):
    temp = pollster.iloc[i,0]
    temp = temp.replace('Public Policy Polling', 'PPP')
    if temp == 'PPP':
        pollster.iloc[i,0] = temp
pollster["score"] = np.nan
gradetoscore =pd.read_csv('gradetoscore.csv')
for i in range(0,len(pollster.iloc[:,0])):
    for j in range(0,len(gradetoscore)):
        if gradetoscore.iloc[j,0]==pollster.iloc[i,1]:
            pollster.iloc[i,2]=gradetoscore.iloc[j,1]

In [6]:
def aggregate(df,pollster):
    poll = pd.Series.to_frame(df['Poll'])
    for i in range (0,len(poll)):
        poll.iloc[i,0] = re.sub("[\(\[].*?[\)\]]", '', poll.iloc[i,0])
        temp = poll.iloc[i,0]
        temp = temp.rstrip()
        poll.iloc[i,0] = temp
        temp = poll.iloc[i,0]
        temp = temp.replace("*","")
        poll.iloc[i,0] = temp
    poll = poll['Poll'].str.split("/", n = 10, expand = True)
    df["score"] = np.nan
    for i in range(0,len(poll.iloc[:,1])):
        max_score = 0
        temp_max = 0
        for j in range(0,len(poll.iloc[1,:])):
            if poll.iloc[i,j] is None:
                break
            else:
                temp = pollster[pollster['Pollster'].str.contains(poll.iloc[i,j], case = False)]
                if len(temp)>0:
                    temp_max = max(temp['score'])
            if max_score < temp_max:
                max_score = temp_max
        df['score'].iloc[i] = max_score
    return df,poll

In [7]:
RCP_direction = pd.read_csv('RCP_direction_final.csv')
direction = RCP_direction.Spread
direction = pd.Series.to_frame(direction)
direction = direction['Spread'].str.split(" ", n = 3, expand = True)
for i in range(0,len(direction.iloc[:,2])):
    if direction.iloc[i,2] is None:
        direction.iloc[i,2] = 0
    else:
        direction.iloc[i,2] = int(re.findall('\d+', direction.iloc[i,2])[0])
    if direction.iloc[1,0]=='Wrong':
        direction.iloc[i,2] = direction.iloc[i,2]*-1
RCP_direction['Spread'] = direction.iloc[:,2]
RCP_direction.head()

,Race/Topic (Click to Sort),Poll,Results,Spread,Date,Year
0,Direction of Country,R2000/Daily Kos (D),"Right Direction 38, Wrong Track 58",-20,2010-04-16,2010
1,Direction of Country,R2000/Daily Kos (D),"Right Direction 39, Wrong Track 57",-18,2010-04-23,2010
2,Direction of Country,Rasmussen Reports,"Right Direction 33, Wrong Track 62",-29,2010-04-26,2010
3,Direction of Country,Democracy Corps (D),"Right Direction 34, Wrong Track 57",-23,2010-04-27,2010
4,Direction of Country,Rasmussen Reports,"Right Direction 33, Wrong Track 61",-28,2010-04-27,2010


In [8]:
RCP_direction,poll = aggregate(RCP_direction.iloc[:],pollster)

/Users/congyang/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
RCP_direction.head()

,Race/Topic (Click to Sort),Poll,Results,Spread,Date,Year,score
0,Direction of Country,R2000/Daily Kos (D),"Right Direction 38, Wrong Track 58",-20,2010-04-16,2010,0.0
1,Direction of Country,R2000/Daily Kos (D),"Right Direction 39, Wrong Track 57",-18,2010-04-23,2010,0.0
2,Direction of Country,Rasmussen Reports,"Right Direction 33, Wrong Track 62",-29,2010-04-26,2010,7.0
3,Direction of Country,Democracy Corps (D),"Right Direction 34, Wrong Track 57",-23,2010-04-27,2010,8.0
4,Direction of Country,Rasmussen Reports,"Right Direction 33, Wrong Track 61",-28,2010-04-27,2010,7.0


In [12]:
RCP_direction.to_csv('RCP_direction_final_538.csv')

In [13]:
RCP_c_approval = pd.read_csv('RCP_c_approval_final.csv')
c_approval = RCP_c_approval.Spread
c_approval = pd.Series.to_frame(c_approval)
c_approval = c_approval['Spread'].str.split(" ", n = 3, expand = True)
for i in range(0,len(c_approval.iloc[:,1])):
    if c_approval.iloc[i,1] is None:
        c_approval.iloc[i,1] = 0
    else:
        c_approval.iloc[i,1] = int(re.findall('\d+', c_approval.iloc[i,1])[0])
    if c_approval.iloc[i,0]=='Disapprove':
        c_approval.iloc[i,1] = c_approval.iloc[i,1]*-1
RCP_c_approval['Spread'] = c_approval.iloc[:,1]
RCP_c_approval.head()

,Race/Topic (Click to Sort),Poll,Results,Spread,Date,Year
0,Congressional Job Approval,Quinnipiac,"Approve 20, Disapprove 71",-51,2010-04-21,2010
1,Congressional Job Approval,FOX News,"Approve 22, Disapprove 68",-46,2010-04-22,2010
2,Congressional Job Approval,NBC News/Wall St. Jrnl,"Approve 21, Disapprove 72",-51,2010-05-12,2010
3,Congressional Job Approval,Gallup,"Approve 21, Disapprove 75",-54,2010-05-14,2010
4,Congressional Job Approval,Associated Press/GfK,"Approve 28, Disapprove 71",-43,2010-05-15,2010


In [14]:
RCP_c_approval,poll = aggregate(RCP_c_approval.iloc[:],pollster)
RCP_c_approval.head()

/Users/congyang/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Race/Topic (Click to Sort),Poll,Results,Spread,Date,Year,score
0,Congressional Job Approval,Quinnipiac,"Approve 20, Disapprove 71",-51,2010-04-21,2010,11.0
1,Congressional Job Approval,FOX News,"Approve 22, Disapprove 68",-46,2010-04-22,2010,12.0
2,Congressional Job Approval,NBC News/Wall St. Jrnl,"Approve 21, Disapprove 72",-51,2010-05-12,2010,11.0
3,Congressional Job Approval,Gallup,"Approve 21, Disapprove 75",-54,2010-05-14,2010,9.0
4,Congressional Job Approval,Associated Press/GfK,"Approve 28, Disapprove 71",-43,2010-05-15,2010,9.0


In [15]:
RCP_c_approval.to_csv('RCP_c_approval_final_538.csv')

In [16]:
RCP_p_approval = pd.read_csv('RCP_p_approval_final.csv')
p_approval = RCP_p_approval.Spread
p_approval = pd.Series.to_frame(p_approval)
p_approval = p_approval['Spread'].str.split(" ", n = 3, expand = True)
for i in range(0,len(p_approval.iloc[:,1])):
    if p_approval.iloc[i,1] is None:
        p_approval.iloc[i,1] = 0
    else:
        p_approval.iloc[i,1] = int(re.findall('\d+', p_approval.iloc[i,1])[0])
    if p_approval.iloc[i,0]=='Disapprove':
        p_approval.iloc[i,1] = p_approval.iloc[i,1]*-1
RCP_p_approval['Spread'] = p_approval.iloc[:,1]
RCP_p_approval.head()

,Race/Topic (Click to Sort),Poll,Results,Spread,Date,Year
0,President Obama Job Approval,Quinnipiac,"Approve 44, Disapprove 46",-2,2010-04-21,2010
1,President Obama Job Approval,FOX News,"Approve 46, Disapprove 46",0,2010-04-22,2010
2,President Obama Job Approval,Gallup,"Approve 50, Disapprove 43",7,2010-04-25,2010
3,President Obama Job Approval,Rasmussen Reports,"Approve 45, Disapprove 54",-9,2010-04-25,2010
4,President Obama Job Approval,Democracy Corps (D),"Approve 48, Disapprove 46",2,2010-04-27,2010


In [17]:
RCP_p_approval,poll = aggregate(RCP_p_approval.iloc[:],pollster)
RCP_p_approval.head()

/Users/congyang/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Race/Topic (Click to Sort),Poll,Results,Spread,Date,Year,score
0,President Obama Job Approval,Quinnipiac,"Approve 44, Disapprove 46",-2,2010-04-21,2010,11.0
1,President Obama Job Approval,FOX News,"Approve 46, Disapprove 46",0,2010-04-22,2010,12.0
2,President Obama Job Approval,Gallup,"Approve 50, Disapprove 43",7,2010-04-25,2010,9.0
3,President Obama Job Approval,Rasmussen Reports,"Approve 45, Disapprove 54",-9,2010-04-25,2010,7.0
4,President Obama Job Approval,Democracy Corps (D),"Approve 48, Disapprove 46",2,2010-04-27,2010,8.0


In [18]:
RCP_p_approval.to_csv('RCP_p_approval_final_538.csv')

In [20]:
#RCP_approval = pd.read_csv('RCP_approval_final.csv')
#approval = RCP_approval.Spread
#approval = pd.Series.to_frame(approval)
#approval = approval['Spread'].str.split(" ", n = 3, expand = True)
#for i in range(0,len(approval.iloc[:,0])):
#    if approval.iloc[i,1] is None:
#        approval.iloc[i,1] = 0
#    else:
#        approval.iloc[i,1] = int(re.findall('\d+', approval.iloc[i,1])[0])
#    if approval.iloc[1,1]=='Disapprove':
#        approval.iloc[i,1] = approval.iloc[i,1]*-1
#RCP_approval['Spread'] = approval.iloc[:,1]
#RCP_approval.head()

In [21]:
#RCP_approval,poll = aggregate(RCP_approval.iloc[:],pollster)
#RCP_approval.head()

In [22]:
#RCP_approval.to_csv('RCP_approval_final_538.csv')

In [23]:
RCP_house = pd.read_csv('RCP_house_final.csv')
house = RCP_house.Spread
house = pd.Series.to_frame(house)
house = house['Spread'].str.split(" ", n = 3, expand = True)
result = RCP_house['Results'].str.split(",", n = 1, expand = True)
#for i in range(0,len(result.iloc[:,0])):
#    if not re.findall('\d+', result.iloc[i,0]):
#        result.iloc[i,0] = '0'
#    result.iloc[i,1] = int(re.findall('\d+', result.iloc[i,1])[0])
#    result.iloc[i,0] = int(re.findall('\d+', result.iloc[i,0])[0])


for i in range(0,len(house.iloc[:,1])):
    if house.iloc[i,2] is None:
        if house.iloc[i,1] is None:
            house.iloc[i,1] = 0
        else:
            house.iloc[i,1] = int(re.findall('\d+', house.iloc[i,1])[0])
    else:
        if house.iloc[i,2] is None:
            house.iloc[i,1] = 0
        else:
            house.iloc[i,1] = int(re.findall('\d+', house.iloc[i,2])[0])
#    if (result.iloc[i,0]<result.iloc[i,1]):
#        house.iloc[i,1] = house.iloc[i,1] *-1
RCP_house['Spread'] = house.iloc[:,1]
RCP_house.head()

,Race/Topic (Click to Sort),Poll,Results,Spread,Date,Year
0,Pennsylvania 12th District Special Election - ...,PPP (D),"Burns 44, Critz 41",3,2010-04-20,2010
1,New Hampshire 1st District - Guinta vs. Shea-P...,PPP (D),"Guinta 46, Shea-Porter 45",1,2010-04-22,2010
2,New Hampshire 2nd District - Bass vs. Swett,PPP (D),"Bass 47, Swett 32",15,2010-04-22,2010
3,New Hampshire 1st District - Guinta vs. Shea-P...,WMUR/UNH,"Guinta 42, Shea-Porter 38",4,2010-04-30,2010
4,New Hampshire 2nd District - Bass vs. Swett,WMUR/UNH,"Bass 44, Swett 27",17,2010-04-30,2010


In [24]:
RCP_house,poll = aggregate(RCP_house.iloc[:],pollster)
RCP_house.head()

/Users/congyang/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Race/Topic (Click to Sort),Poll,Results,Spread,Date,Year,score
0,Pennsylvania 12th District Special Election - ...,PPP (D),"Burns 44, Critz 41",3,2010-04-20,2010,9.0
1,New Hampshire 1st District - Guinta vs. Shea-P...,PPP (D),"Guinta 46, Shea-Porter 45",1,2010-04-22,2010,9.0
2,New Hampshire 2nd District - Bass vs. Swett,PPP (D),"Bass 47, Swett 32",15,2010-04-22,2010,9.0
3,New Hampshire 1st District - Guinta vs. Shea-P...,WMUR/UNH,"Guinta 42, Shea-Porter 38",4,2010-04-30,2010,0.0
4,New Hampshire 2nd District - Bass vs. Swett,WMUR/UNH,"Bass 44, Swett 27",17,2010-04-30,2010,0.0


In [25]:
RCP_house.to_csv('RCP_house_final_538.csv')

In [26]:
RCP_govenor = pd.read_csv('RCP_governor_final.csv')
govenor = RCP_govenor.Spread
govenor = pd.Series.to_frame(govenor)
govenor = govenor['Spread'].str.split(" ", n = 3, expand = True)
#result = RCP_govenor['Results'].str.split(",", n = 1, expand = True)
#for i in range(0,len(result.iloc[:,0])):
#    if not re.findall('\d+', result.iloc[i,0]):
#        result.iloc[i,0] = '0'
#    result.iloc[i,1] = int(re.findall('\d+', result.iloc[i,1])[0])
#    result.iloc[i,0] = int(re.findall('\d+', result.iloc[i,0])[0])

for i in range(0,len(govenor.iloc[:,1])):
    if govenor.iloc[i,2] is None:
        if govenor.iloc[i,1] is None:
            govenor.iloc[i,1] = 0
        else:
            govenor.iloc[i,1] = int(re.findall('\d+', govenor.iloc[i,1])[0])
    else:
        if govenor.iloc[i,2] is None:
            govenor.iloc[i,1] = 0
        else:
            govenor.iloc[i,1] = int(re.findall('\d+', govenor.iloc[i,2])[0])
#    if (result.iloc[i,0]<result.iloc[i,1]):
#        govenor.iloc[i,1] = govenor.iloc[i,1] *-1
RCP_govenor['Spread'] = govenor.iloc[:,1]
RCP_govenor.head()

,Race/Topic (Click to Sort),Poll,Results,Spread,Date,Year
0,Colorado Governor - McInnis vs. Hickenlooper,Rasmussen Reports,"Hickenlooper 42, McInnis 48",6,2010-04-16,2010
1,Arizona Governor - Republican Primary,Rasmussen Reports,"Brewer 26, Mills 18, Martin 12, Munger 14",8,2010-04-17,2010
2,New York Governor - Republican Primary,Siena,"Lazio 29, Levy 15, Paladino 13",14,2010-04-18,2010
3,New York Governor - Levy vs. Cuomo,Siena,"Cuomo 58, Levy 23",35,2010-04-18,2010
4,New York Governor - Lazio vs. Cuomo,Siena,"Cuomo 61, Lazio 24",37,2010-04-18,2010


In [27]:
RCP_govenor,poll = aggregate(RCP_govenor.iloc[:],pollster)
RCP_govenor.head()

/Users/congyang/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Race/Topic (Click to Sort),Poll,Results,Spread,Date,Year,score
0,Colorado Governor - McInnis vs. Hickenlooper,Rasmussen Reports,"Hickenlooper 42, McInnis 48",6,2010-04-16,2010,7.0
1,Arizona Governor - Republican Primary,Rasmussen Reports,"Brewer 26, Mills 18, Martin 12, Munger 14",8,2010-04-17,2010,7.0
2,New York Governor - Republican Primary,Siena,"Lazio 29, Levy 15, Paladino 13",14,2010-04-18,2010,12.0
3,New York Governor - Levy vs. Cuomo,Siena,"Cuomo 58, Levy 23",35,2010-04-18,2010,12.0
4,New York Governor - Lazio vs. Cuomo,Siena,"Cuomo 61, Lazio 24",37,2010-04-18,2010,12.0


In [28]:
RCP_govenor.to_csv('RCP_governor_final_538.csv')

In [ ]:
RCP_senate = pd.read_csv('RCP_senate_final.csv')
senate = RCP_senate.Spread
senate = pd.Series.to_frame(senate)
senate = senate['Spread'].str.split(" ", n = 3, expand = True)
result = RCP_senate['Results'].str.split(",", n = 100, expand = True)
#for i in range(0,len(result.iloc[:,0])):
#    if not re.findall('\d+', result.iloc[i,0]):
#        result.iloc[i,0] = '0'
#    result.iloc[i,1] = int(re.findall('\d+', result.iloc[i,1])[0])
#    result.iloc[i,0] = int(re.findall('\d+', result.iloc[i,0])[0])


for i in range(0,len(senate.iloc[:,1])):
    if senate.iloc[i,2] is None:
        if senate.iloc[i,1] is None:
            senate.iloc[i,1] = 0
        else:
            senate.iloc[i,1] = int(re.findall('\d+', senate.iloc[i,1])[0])
    else:
        if senate.iloc[i,2] is None:
            senate.iloc[i,1] = 0
        else:
            senate.iloc[i,1] = int(re.findall('\d+', senate.iloc[i,2])[0])
#    if (result.iloc[i,0]<result.iloc[i,1]):
#        senate.iloc[i,1] = senate.iloc[i,1] *-1
RCP_senate['Spread'] = senate.iloc[:,1]
RCP_senate.head()

,Race/Topic (Click to Sort),Poll,Results,Spread,Date,Year
0,Florida Senate - Rubio vs. Meek vs. Crist,Quinnipiac,"Crist 32, Rubio 30, Meek 24",2,2010-04-15,2010
1,Arkansas Senate - Boozman vs. Lincoln,R2000/Daily Kos (D),"Boozman 50, Lincoln 43",7,2010-04-15,2010
2,Arkansas Senate - Boozman vs. Halter,R2000/Daily Kos (D),"Boozman 48, Halter 41",7,2010-04-15,2010
3,Florida Senate - Rubio vs. Meek,Quinnipiac,"Rubio 42, Meek 38",4,2010-04-15,2010
4,Florida Senate - Republican Primary,Quinnipiac,"Rubio 56, Crist 33",23,2010-04-15,2010


In [ ]:
RCP_senate,poll = aggregate(RCP_senate.iloc[:],pollster)
RCP_senate.head()

/Users/congyang/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
RCP_senate.to_csv('RCP_senate_final_538.csv')

In [ ]:
RCP_generic = pd.read_csv('RCP_generic_final.csv')
generic = RCP_generic.Spread
generic = pd.Series.to_frame(generic)
generic = generic['Spread'].str.split(" ", n = 3, expand = True)
for i in range(0,len(generic.iloc[:,1])):
    if generic.iloc[i,1] is None:
        generic.iloc[i,1] = 0
    else:
        generic.iloc[i,1] = int(re.findall('\d+', generic.iloc[i,1])[0])
    if generic.iloc[i,0]=='Republicans':
        generic.iloc[i,1] = generic.iloc[i,1]*-1
RCP_generic['Spread'] = generic.iloc[:,1]
RCP_generic.head()

In [ ]:
RCP_generic,poll = aggregate(RCP_generic.iloc[:],pollster)
RCP_generic.head()

In [ ]:
RCP_generic.to_csv('RCP_generic_final_538.csv')

In [ ]:
#RCP_senate_DemRec = pd.read_csv('RCP_senate_DemRec.csv')
#senate_DemRec = RCP_senate_DemRec.Spread
#senate_DemRec = pd.Series.to_frame(senate_DemRec)
#senate_DemRec = senate_DemRec['Spread'].str.split(" ", n = 3, expand = True)
#for i in range(0,len(senate_DemRec.iloc[:,1])):
#    if senate_DemRec.iloc[i,2] is None:
#        if senate_DemRec.iloc[i,1] is None:
#            senate_DemRec.iloc[i,1] = 0
#        else:
#            senate_DemRec.iloc[i,1] = int(re.findall('\d+', senate_DemRec.iloc[i,1])[0])
#    else:
#        if senate_DemRec.iloc[i,2] is None:
#            senate_DemRec.iloc[i,1] = 0
#        else:
#            senate_DemRec.iloc[i,1] = int(re.findall('\d+', senate_DemRec.iloc[i,2])[0])

#RCP_senate_DemRec['Spread'] = senate_DemRec.iloc[:,1]
#RCP_senate_DemRec.head()

In [ ]:
#RCP_senate_DemRec,poll = aggregate(RCP_senate_DemRec.iloc[:],pollster)
#RCP_senate_DemRec.head()

In [ ]:
#RCP_senate_DemRec.to_csv('RCP_senate_DemRec.csv')